# Data Cleaning and Transformation Guide
## Introduction

This Jupyter Notebook provides a step-by-step guide for cleaning and transforming data using Python and the Pandas library. In this example, we will work with a JSON dataset, performing various operations to prepare it for analysis and storage. The goal is to make the data more consistent, accurate, and ready for use in data analysis or storage in a data warehouse or SQL database.

---


## Step 1: Import Necessary Libraries

In [ ]:
import pandas as pd

## Step 2: Load the Data

In [ ]:
data = pd.read_json('your_data.json')

## Step 3: Data Type Conversion and Currency Conversion

Remove symbols and convert certain columns to numeric. Also, convert pricing to both USD and GBP.

In [ ]:
data['Current Price'] = data['Current Price'].str.replace('[\$,]', '', regex=True).astype(float)
data['Previous Price'] = data['Previous Price'].str.replace('[\$,]', '', regex=True).astype(float)
data['Price Per Each'] = data['Price Per Each'].str.replace('[\$,]', '', regex=True).astype(float)

exchange_rate = 1.22169  # 1 GBP = 1.22169 USD
data['Price USD'] = data['Current Price']  # USD price
data['Price GBP'] = data['Current Price'] / exchange_rate  # GBP price


## Step 4: Create Price Per Each or Price Per Unit column

Clean the "Price Per Each" column and extract the numeric values.

In [ ]:
data['Price Per Each'] = data['Price Per Each'].str.extract(r'(\d+\.\d+)').astype(float)

## Step 5: Handling "about" Prices

Clean the "about" prices and extract the numeric values. 

In [ ]:
data['Current Price'] = data['Current Price'].str.replace('about', '', case=False)
data['Previous Price'] = data['Previous Price'].str.replace('about', '', case=False)

## Step 6: Category as a Dimension

Extract unique categories as a separate dimension.

In [ ]:
category_dimension = data[['Category']].drop_duplicates().reset_index(drop=True)

## Step 7: Handling Duplicate IDs

Make product IDs unique by appending a count to them.

In [ ]:
data['Product ID'] = data.groupby('Product ID').cumcount().astype(str) + '_' + data['Product ID']

## Step 8: Related Table as a Dimension (Unit of Measure)

Clean the "Category" column to remove any non-alphabetical characters.

In [ ]:
data['Category'] = data['Category'].str.extract(r'([a-zA-Z\s]+)')

## Step 9: Handling Missing Values

Fill any missing values with empty strings.

In [ ]:
data.fillna('', inplace=True)

## Step 10: Removing Duplicates

Remove duplicate rows, keeping the first occurrence.

In [ ]:
data.drop_duplicates(keep='first', inplace=True)

## Step 11: Removing Special Characters

Remove special characters from the "Name" column.

In [ ]:
data['Name'] = data['Name'].str.replace('[^a-zA-Z0-9\s]+', '', regex=True)

## Step 12: Decimal Placement

Round numeric columns to two decimal places.

In [ ]:
data['Current Price'] = data['Current Price'].round(2)
data['Previous Price'] = data['Previous Price'].round(2)
data['Price Per Each'] = data['Price Per Each'].round(2)
data['Price USD'] = data['Price USD'].round(2)
data['Price GBP'] = data['Price GBP'].round(2)

## Step 15: Save the Cleaned Data

Save the cleaned data to JSON files for import into a data warehouse or SQL database.

In [ ]:
data.to_json('cleaned_data.json', orient='records')
category_dimension.to_json('category_dimension.json', orient='records')